In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk 

In [2]:
df = pd.read_csv("L:\\ML-Assignment\\training_data.csv")
df.astype({"joined_text" : "string" , "class" : "string" })

C:\Users\CHEN KANG\AppData\Local\Temp\ipykernel_34748\246959809.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("L:\\ML-Assignment\\training_data.csv")


,joined_text,class
0,is upset that he can not update his Facebook b...,0
1,I dived many times for the ball Managed to sav...,0
2,my whole body feels itchy and like its on fire,0
3,no its not behaving at all i am mad why am i h...,0
4,not the whole crew,0
...,...,...
1828592,I did something today I went sledding with my ...,non-suicide
1828593,If you do not like rock then your not going to...,non-suicide
1828594,You how you can tell i have so many friends an...,non-suicide
1828595,pee probably tastes like salty tea can someone...,non-suicide


In [3]:
#split the data into 2 classes according to the sentiment levels 
splits = list(df.groupby("class"))


negative = splits[0][1]
positive1 = splits[1][1]
positive2 = splits[2][1]
suicidal = splits[4][1]

positive = pd.concat([positive1, positive2], axis = 0)

In [4]:
positive.sample(frac = 1)
negative.sample(frac = 1)
suicidal.sample(frac = 1)

,joined_text,class
1691217,I should be grateful for this lifeI know I sho...,suicide
1680230,Even on a normal good day I still think about ...,suicide
1641348,My dad killed himself last nightI feel numb an...,suicide
1712103,Is this legitIn comments is the link If I boug...,suicide
1626613,damaged the wall with a knife I really think I...,suicide
...,...,...
1604488,Feeling SuicidalHello everyone i did not know ...,suicide
1665846,I intend toOver a year ago I made an irreversi...,suicide
1646654,at an impassejust when things started to look ...,suicide
1711989,just really really want to die alreadyso over ...,suicide


In [5]:
#randomly select 50000 features from each class 
import random as rand

number = rand.randint(0,500000)
new_positive = positive[number : number + 5000]

number = rand.randint(0,500000)
new_negative = negative[number: number + 5000]

number = rand.randint(0,75000)
new_suicidal = suicidal[number: number+ 5000]

In [6]:
#concat 3 classes into 1 dataframe
df_concat = pd.concat([new_positive, new_negative, new_suicidal], axis = 0)

#shuffle the sequence of the dataframe
df_concat = df_concat.sample(frac = 1)

In [7]:
x_val = df_concat["joined_text"]
y_val = df_concat["class"]

In [8]:
np_x_val = np.asarray(x_val.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
np_y_val = np.asarray(y_val.to_frame().to_numpy(dtype=np.string_).astype(np.string_))

In [9]:
def create_label(y):
    counter = 0
    result = np.array([[]])
    if y[0] == np.string_("0"):
        result = np.array([[1,0,0]])
    elif y[0] == np.string_("4"):
        result = np.array([[0,1,0]])
    elif y[0] == np.string_("suicide"):
        result = np.array([[0,0,1]])
    
    for i in y: 
        if counter > 0:
            if i == np.string_("0"):
                result = np.append(result, np.array([[1,0,0]]), axis = 0)
            elif i == np.string_("4"):
                result = np.append(result, np.array([[0,1,0]]), axis = 0)
            elif i == np.string_("suicide"):
                result = np.append(result, np.array([[0,0,1]]), axis = 0)
        counter+=1
    

    return result

np_y_val = create_label(np_y_val)

In [21]:
model_dir = "L:\\ML-Assignment\\Model\\Model_06"

In [22]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Dense, TextVectorization, LSTM
from sklearn.metrics import accuracy_score

In [1]:
model = load_model(model_dir)

print("Loaded")

NameError: name 'load_model' is not defined

In [13]:
pred = model.predict(np_x_val)

469/469 [==============================] - 82s 158ms/step


In [14]:
def change_output(y):
    output = np.array([])
    
    if y[0][0] == y[0].max():
        output = np.array([0])
    elif y[0][1] == y[0].max():
        output = np.array([1])
    elif y[0][2] == y[0].max():
        output = np.array([2])
    
    i = 1
    
    while i < pred.size/3: 
        if y[i][0] == y[i].max():
            output = np.append(output, np.array([0]), axis = 0)
        elif y[i][1] == y[i].max():
            output = np.append(output, np.array([1]), axis = 0)
        elif y[i][2] == y[i].max():
            output = np.append(output, np.array([2]), axis = 0)
        i+=1 
            
    return output

y_pred = change_output(pred)
y_true = change_output(np_y_val)

In [15]:
from sklearn.metrics import hamming_loss
hamming_loss(y_true, y_pred)

0.20193333333333333

In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np_y_val, pred)

0.9222156233333334

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.7980666666666667

In [18]:
from sklearn.metrics import jaccard_score
jaccard_score(y_true, y_pred, average = "weighted")

0.6830860219346594

In [19]:
from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average = "weighted")

0.79733784973623

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred, labels = [0, 1, 2])

array([[3573, 1346,   81],
       [1443, 3478,   79],
       [  52,   28, 4920]], dtype=int64)

In [ ]:
''